In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../pruebillas.ipynb'), '..'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))

In [2]:
import yfinance as yf
import pandas as pd
from backtesting import Backtest, Strategy

import pandas as pd
import talib as ta

pd.set_option('display.float_format', lambda x: '%.6f' % x)

In [5]:
vix = yf.Ticker("^VIX").history(interval='1h', period='1y')
vix

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2023-09-25 13:00:00-05:00,17.290001,17.420000,17.020000,17.320000,0,0.000000,0.000000
2023-09-25 14:00:00-05:00,17.219999,17.639999,16.820000,16.820000,0,0.000000,0.000000
2023-09-25 15:00:00-05:00,16.850000,16.920000,16.790001,16.889999,0,0.000000,0.000000
2023-09-26 02:00:00-05:00,18.030001,18.150000,17.700001,17.719999,0,0.000000,0.000000
2023-09-26 03:00:00-05:00,18.030001,18.090000,17.280001,17.469999,0,0.000000,0.000000
...,...,...,...,...,...,...,...
2024-09-25 09:00:00-05:00,15.300000,15.350000,15.190000,15.260000,0,0.000000,0.000000
2024-09-25 10:00:00-05:00,15.270000,15.350000,15.170000,15.170000,0,0.000000,0.000000
2024-09-25 11:00:00-05:00,15.180000,15.440000,15.170000,15.430000,0,0.000000,0.000000


In [3]:
# dji = yf.Ticker("^DJI").history(period="max")
vix = yf.Ticker("^VIX").history(interval='1h', period='max')
vix

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2022-09-26 02:00:00-05:00,31.740000,31.840000,31.299999,31.480000,0,0.000000,0.000000
2022-09-26 03:00:00-05:00,31.480000,32.090000,31.280001,31.670000,0,0.000000,0.000000
2022-09-26 04:00:00-05:00,31.639999,32.330002,31.610001,32.320000,0,0.000000,0.000000
2022-09-26 05:00:00-05:00,32.330002,32.880001,32.320000,32.369999,0,0.000000,0.000000
2022-09-26 06:00:00-05:00,32.330002,32.630001,32.270000,32.320000,0,0.000000,0.000000
...,...,...,...,...,...,...,...
2024-09-24 08:00:00-05:00,15.880000,15.920000,15.710000,15.870000,0,0.000000,0.000000
2024-09-24 09:00:00-05:00,15.810000,16.670000,15.810000,16.040001,0,0.000000,0.000000
2024-09-24 10:00:00-05:00,16.040001,16.040001,15.580000,15.680000,0,0.000000,0.000000


In [4]:
from datetime import timedelta
import pytz
import MetaTrader5 as mt5
from datetime import datetime, timedelta


print("MetaTrader5 package author: ", mt5.__author__)
print("MetaTrader5 package version: ", mt5.__version__)

# Establecer conexión con el terminal de MetaTrader 5
if not mt5.initialize():
    raise Exception("initialize() failed, error code =", mt5.last_error())

# Establecer la zona horaria a UTC
timezone = pytz.timezone("Etc/UTC")

# Crear objetos 'datetime' en zona horaria UTC
now = datetime.now(tz=timezone)
date_from = now - timedelta(days=20) - timedelta(days=500) 
            
# Obtener las tasas históricas
rates = mt5.copy_rates_range('USTECm', mt5.TIMEFRAME_H1, date_from, now)

# Crear DataFrame con las tasas
df = pd.DataFrame(rates)

# Convertir el tiempo de segundos a formato datetime
df['time'] = pd.to_datetime(df['time'], unit='s')


# Renombrar columnas para el ticker principal
df = df.rename(columns={
    'time': 'Date',
    'open': 'Open',
    'high': 'High',
    'low': 'Low',
    'close': 'Close',
    'tick_volume': 'Volume'
}).set_index('Date')


# Cerrar la conexión con MetaTrader 5
mt5.shutdown()

MetaTrader5 package author:  MetaQuotes Ltd.
MetaTrader5 package version:  5.0.4288


True

In [5]:
vix.rename(
    columns={'Close':'VixClose'}, inplace=True
)

In [6]:
df.index = df.index.tz_localize('UTC').tz_convert('UTC')
vix.index = vix.index.tz_convert('UTC')

In [7]:
print(df.tail(1).index)
print(vix.tail(1).index)

DatetimeIndex(['2024-09-24 17:00:00+00:00'], dtype='datetime64[ns, UTC]', name='Date', freq=None)
DatetimeIndex(['2024-09-24 17:00:00+00:00'], dtype='datetime64[ns, UTC]', name='Datetime', freq=None)


In [8]:
merge = pd.merge(
    df, 
    vix[['VixClose']], 
    left_index=True, 
    right_index=True
)

merge

,Open,High,Low,Close,Volume,spread,real_volume,VixClose
2023-04-24 07:00:00+00:00,12947.970000,12962.540000,12935.190000,12956.820000,1799,421,0,18.070000
2023-04-24 08:00:00+00:00,12956.890000,12963.860000,12941.630000,12951.790000,1480,350,0,17.980000
2023-04-24 09:00:00+00:00,12951.650000,12997.940000,12947.640000,12987.870000,1549,348,0,17.700001
2023-04-24 10:00:00+00:00,12987.820000,12996.210000,12975.620000,12996.210000,1331,353,0,17.730000
2023-04-24 11:00:00+00:00,12996.820000,13013.180000,12992.260000,12996.100000,1565,362,0,17.510000
...,...,...,...,...,...,...,...,...
2024-09-24 13:00:00+00:00,19904.080000,19943.760000,19873.960000,19881.340000,4877,514,0,15.870000
2024-09-24 14:00:00+00:00,19882.490000,19882.490000,19744.930000,19865.350000,7773,510,0,16.040001
2024-09-24 15:00:00+00:00,19866.230000,20005.390000,19860.800000,19978.430000,4939,501,0,15.680000
2024-09-24 16:00:00+00:00,19978.960000,20004.710000,19952.280000,19953.540000,4299,504,0,15.720000


In [9]:
from backbone.trader_bot import TraderBot


def ll_hh_indicator(close, window=None):
    if type(close) != pd.Series:
        close = close.s

    rolling_min = close.rolling(window=window, min_periods=1).min()
    is_lower_low = close == rolling_min

    rolling_max = close.rolling(window=window, min_periods=1).max()
    is_higher_high = close == rolling_max

    return is_lower_low, is_higher_high


class VixRsi(Strategy):
    vix_percentage_above_sma = 0.05
    ll_hh_window = 5
    rsi_threshold = 50
    sma_period = 200
    vix_sma_period = 10
    rsi_period = 2
    
    def init(self):
        self.rsi = self.I(ta.RSI, self.data.Close, timeperiod=self.rsi_period)
        self.sma = self.I(ta.SMA, self.data.Close, timeperiod=self.sma_period)
        self.vix_sma = self.I(ta.SMA, self.data.VixClose, timeperiod=self.vix_sma_period)
        self.lower_low, self.higher_high = self.I(ll_hh_indicator, self.data.Close, window=self.ll_hh_window)

    def next(self):
        actual_close = self.data.Close[-1]

        if self.position:
            first_trade = self.trades[0]
            today = self.data.index[-1].tz_convert('UTC')
            time_in_position = (today - first_trade.entry_time.tz_convert('UTC'))
            time_in_position = time_in_position.days

            if self.position.is_long:
                if self.higher_high:
                    self.position.close()

        else:
            cum_rsi = self.rsi[-1] + self.rsi[-2]

            vix_sma_value = self.vix_sma[-1]
            vix_close = self.data.VixClose[-1]
            vix_above_sma = vix_close > (vix_sma_value * (1 + self.vix_percentage_above_sma))

            if vix_above_sma and cum_rsi <= self.rsi_threshold and actual_close > self.sma[-1]:
                self.buy(size=1)
                
    def next_live(self, trader:TraderBot):
        actual_close = self.data.Close[-1]

        positions = trader.get_open_positions()
        if positions and positions[-1] == mt5.ORDER_TYPE_BUY:
            if self.higher_high:
                trader.close_order(positions[-1])

        else:
            cum_rsi = self.rsi[-1] + self.rsi[-2]

            vix_sma_value = self.vix_sma[-1]
            vix_close = self.data.VixClose[-1]
            vix_above_sma = vix_close > (vix_sma_value * (1 + self.vix_percentage_above_sma))

            if vix_above_sma and cum_rsi <= self.rsi_threshold and actual_close > self.sma:
                info_tick = trader.get_info_tick()
                price = info_tick.ask
                
                trader.open_order(
                    ticker=self.ticker, 
                    lot=self.lot_size, 
                    type_='buy',
                    price=price
                )

In [10]:
# start_date = '2022-01-01'
# end_date = '2022-02-01'


bt = Backtest(
    merge, 
    VixRsi, 
    cash=100000, 
    commission=7e-4
)

stats = bt.run()


stats

Start                     2023-04-24 07:00...
End                       2024-09-24 17:00...
Duration                    519 days 10:00:00
Exposure Time [%]                    3.120000
Equity Final [$]                100728.513142
Equity Peak [$]                 100728.513142
Return [%]                           0.728513
Buy & Hold Return [%]               54.200954
Return (Ann.) [%]                    0.496948
Volatility (Ann.) [%]                0.396759
Sharpe Ratio                         1.252520
Sortino Ratio                        2.308332
Calmar Ratio                         1.775369
Max. Drawdown [%]                   -0.279913
Avg. Drawdown [%]                   -0.082240
Max. Drawdown Duration      164 days 02:00:00
Avg. Drawdown Duration       33 days 12:00:00
# Trades                                   24
Win Rate [%]                        83.333333
Best Trade [%]                       1.194868
Worst Trade [%]                     -1.054048
Avg. Trade [%]                    

In [11]:
bt.plot()

c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\mtvenv\lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p1747', ...)

In [12]:
import numpy as np
def  optim_func(series):
    return (series['Return [%]'] /  (1 + (-1*series['Max. Drawdown [%]']))) * np.log(1 + series['# Trades'])

In [13]:
from utils import walk_forward

lookback_bars = 480
validation_bars = 100
warmup_bars = 300

params = {
    'll_hh_window': [5, 7, 10],
    'vix_percentage_above_sma': [0.01, 0.02, 0.03, 0.04, 0.05],
    'maximize': optim_func
}

stats = walk_forward(
    VixRsi,
    merge, 
    lookback_bars=lookback_bars,
    validation_bars=validation_bars,
    warmup_bars=warmup_bars, 
    params=params,
    commission=7e-4, 
    margin=1/30, 
    cash=15_000
)

  0%|          | 0/15 [00:00<?, ?it/s]

equity final: 15232.173423000004


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\backtesting\utils.py:75: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt_validation = Backtest(
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\backtesting\utils.py:58: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt_training = Backtest(


  0%|          | 0/15 [00:00<?, ?it/s]

equity final: 15511.530289000017


...

equity final: 18007.125867000155


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\backtesting\utils.py:75: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt_validation = Backtest(
c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\backtesting\utils.py:58: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt_training = Backtest(


  0%|          | 0/15 [00:00<?, ?it/s]

equity final: 18286.85165400016


c:\Users\SaidJ\OneDrive\Documentos\projects\forex_ml_bot\forex_ml_bot\backtesting\utils.py:75: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt_validation = Backtest(


In [14]:
from utils import plot_full_equity_curve

plot_full_equity_curve(
    merge, 
    stats, 
    warmup_bars=warmup_bars,
    lookback_bars=lookback_bars, 
    overlay_price=True
)

In [15]:
mt5.TIMEFRAME_H1

16385